In [6]:
!pip install transformers datasets evaluate rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=c9ccf0a365917e8353822733bd6bbf8ae7867b67cd9c7339550fc5c233b3f69a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [2]:
!wget https://github.com/otaviofcoletti/Projeto-Pr-tico-de-PLN/raw/main/data.zip

--2024-05-29 14:28:06--  https://github.com/otaviofcoletti/Projeto-Pr-tico-de-PLN/raw/main/data.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/otaviofcoletti/Projeto-Pr-tico-de-PLN/main/data.zip [following]
--2024-05-29 14:28:06--  https://raw.githubusercontent.com/otaviofcoletti/Projeto-Pr-tico-de-PLN/main/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16982989 (16M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  16.20M  --.-KB/s    in 0.1s    

2024-05-29 14:28:06 (114 MB/s) - ‘data.zip’ saved [16982989/16982989]



In [3]:
!unzip "/content/data.zip"

Archive:  /content/data.zip
   creating: test/
   creating: test/ects/
  inflating: test/ects/AAN_q3_2021.txt  
  inflating: test/ects/AAN_q4_2020.txt  
  inflating: test/ects/AAP_q4_2020.txt  
  inflating: test/ects/AAT_q1_2021.txt  
  inflating: test/ects/AAT_q4_2020.txt  
  inflating: test/ects/AA_q3_2021.txt  
  inflating: test/ects/ABM_q3_2021.txt  
  inflating: test/ects/ACC_q3_2020.txt  
  inflating: test/ects/ADC_q3_2021.txt  
  inflating: test/ects/ADM_q1_2021.txt  
  inflating: test/ects/AEL_q3_2021.txt  
  inflating: test/ects/AFL_q4_2020.txt  
  inflating: test/ects/AIR_q1_2022.txt  
  inflating: test/ects/AIT_q2_2020.txt  
  inflating: test/ects/AJG_q3_2021.txt  
  inflating: test/ects/ALB_q3_2021.txt  
  inflating: test/ects/ALB_q4_2021.txt  
  inflating: test/ects/ALE_q1_2021.txt  
  inflating: test/ects/ALG_q2_2021.txt  
  inflating: test/ects/ALL_q2_2021.txt  
  inflating: test/ects/AME_q1_2021.txt  
  inflating: test/ects/AMG_q2_2021.txt  
  inflating: test/ects/AMG_q

In [7]:
import os
from datasets import Dataset, DatasetDict

def load_text_and_summaries(text_dir, summary_dir):
    data = {"text": [], "summary": [], "title": []}

    # List all files in the text directory
    text_files = os.listdir(text_dir)
    summary_files = os.listdir(summary_dir)

    # Ensure both directories have the same files
    assert set(text_files) == set(summary_files), "Text and summary directories must have matching files"

    for filename in text_files:
        # Read the text file
        with open(os.path.join(text_dir, filename), 'r', encoding='utf-8') as text_file:
            text = text_file.read()

        # Read the summary file
        with open(os.path.join(summary_dir, filename), 'r', encoding='utf-8') as summary_file:
            summary = summary_file.read()

        # Extract the title from the filename (assuming the title is the filename without extension)
        title = os.path.splitext(filename)[0]

        # Append to the data dictionary
        data["text"].append(text)
        data["summary"].append(summary)
        data["title"].append(title)

    return data

def create_dataset_dict(train_dir, test_dir):
    dataset_dict = {}

    for split in ['train', 'test', 'val']:
        text_dir = os.path.join(train_dir if split == 'train' else test_dir, 'ects')
        summary_dir = os.path.join(train_dir if split == 'train' else test_dir, 'gt_summaries')

        data = load_text_and_summaries(text_dir, summary_dir)
        dataset_dict[split] = Dataset.from_dict(data)

    return DatasetDict(dataset_dict)

# Define the directories
train_directory = 'train'
test_directory = 'test'
test_directory = 'val'

# Create the DatasetDict
dataset_dict = create_dataset_dict(train_directory, test_directory)

# Print the dataset_dict to check
print(dataset_dict)


DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 1681
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 249
    })
    val: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 249
    })
})


In [ ]:
dataset_dict['test'][0]

In [9]:
from transformers import AutoTokenizer

checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [10]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    print(inputs)
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [11]:
tokenized_ects = dataset_dict.map(preprocess_function, batched=True)


Map:   0%|          | 0/1681 [00:00<?, ? examples/s]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Map:   0%|          | 0/249 [00:00<?, ? examples/s]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Map:   0%|          | 0/249 [00:00<?, ? examples/s]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:
#(tokenized_ects['train'])

Dataset({
    features: ['text', 'summary', 'title', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1681
})

In [13]:
#(tokenized_ects['train'][0]['labels'])

[12669,
 13,
 3,
 32,
 75,
 235,
 1152,
 11,
 505,
 6170,
 13,
 3,
 1824,
 519,
 6503,
 3170,
 11,
 3938,
 2055,
 43,
 118,
 1866,
 38,
 13,
 3,
 32,
 75,
 235,
 1152,
 11558,
 6503,
 5,
 668,
 5988,
 13,
 792,
 16801,
 130,
 539,
 11,
 2699,
 6,
 38,
 13,
 3,
 32,
 75,
 235,
 1152,
 11558,
 6503,
 6,
 3,
 390,
 30,
 2041,
 1601,
 1247,
 3170,
 5,
 1]

In [14]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [15]:
import evaluate

rouge = evaluate.load("rouge")

In [16]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [17]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [26]:
!pip install accelerate -U

In [23]:
!pip install transformers[torch]

In [27]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_ects_model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ects["train"],
    eval_dataset=tokenized_ects["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`